### Set up the analysis notebook

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate

!git clone http://github.com/tjbarnum13/Fourier-DVR-1D.git
sys.path.insert(0,'/content/Fourier-DVR-1D')

from fourier_DVR_1D import Domain_Fourier_DVR_1D

In [2]:
# some useful conversion factors
hartreetowavenumber=219474.63136320
atomicmasstoelectronmass=1822.89

### Import IRC data and construct the potential energy curve

In [3]:
# import data
_ , e1, x1 = np.genfromtxt('', unpack=True) # IRC starting from transition state
_ , e2, x2 = np.genfromtxt('', unpack=True) # IRC starting from high-energy wing

In [ ]:
# energies of the high-energy wing and transition state
e_wing=
e_TS=
# shift the IRC points by the energy offset
e1=e1+e_wing
e2=e2+e_TS
x1=x2[-1]+x1[-1]-x1
# piece the high-energy wing and transition state IRC together (deleting the repeated point)
e=np.concatenate((e2[:-1],np.flip(e1)))
x=np.concatenate((x2[:-1],np.flip(x1)))
# copy, reflect, and piece togehter the two halves of the PEC
e=np.concatenate((np.flip(e),e[1:]))
x=np.concatenate((-np.flip(x),x[1:]))
# plot the IRC data to make sure we did this correctly!
plt.figure()
plt.plot(x,e,'ko')

In [ ]:
# generate an interpolated PEC over a finer grid
X=np.linspace(x[0],x[-1],1000)
V=scipy.interpolate.CubicSpline(x,e)
# plot the new potential
plt.figure()
plt.plot(X,V(X),'-k')

### Numerical integration of Schrodinger equation with PEC to generate eigenenergies and vibrational wave functions

In [6]:
# settings
x_min = -5
x_max = 5
m=atomicmasstoelectronmass
n_DVR = 1001 # integration grid
n_g = 1001 # plotting grid
scale_wfn=0.001 # scaling factor for plotting
n_plot=6 # number of wave functions to solve and plot (you might want to change this)

In [7]:
# set up the integration grid
domain = Domain_Fourier_DVR_1D(x_min, x_max, n_DVR)

In [8]:
# solve
E, E_four = domain.solve(m, V)

# evaluate eigenstates on grid
x = np.linspace(x_min, x_max, n_g)
psi_x = domain.grid(x, E_four[:,:n_plot])

In [ ]:
# plot eigenstates and energies
plt.figure(figsize=(10, 6))

for i in range(n_plot):
    plt.plot([x[0], x[-1]], [E[i], E[i]], '-', color='gray', lw=1.0)

plt.plot(x, V(x), 'k-', lw=2)

for i in range(n_plot):
    plt.plot(x, scale_wfn * psi_x[i] + E[i], lw=1.0)

plt.xlabel('reaction coordinate')
plt.ylabel('energy')
plt.tight_layout()

plt.show()